# Simple Neural Network to learn XOR in PyTorch

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

In [3]:
class NN(nn.Module):
  def __init__(self):
    super(NN, self).__init__()
    self.l1_linear = nn.Linear(2, 4, bias=False)
    self.l2_linear = nn.Linear(4, 1, bias=False)
  
  def forward(self, x):
    l1 = self.l1_linear(x)
    out = F.sigmoid(self.l1_linear(x))
    out = F.sigmoid(self.l2_linear(out))
    return out

xor_nn = NN()
optimizer = torch.optim.Adam(xor_nn.parameters(), lr=1e-3, weight_decay=1e-5)

# prepare the training data
x_in = Variable(torch.FloatTensor([[1,1],[0,0],[1,0],[0,1]]))
y_out = Variable(torch.FloatTensor([[0],[0],[1],[1]]))

In [4]:
for i in range(20000):
  predict = xor_nn(x_in)
  loss = F.smooth_l1_loss(predict,y_out)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if (i+1)%5000 == 0:
    print('i: %d, loss: %.4f'%(i+1,loss.data[0]))

i: 5000, loss: 0.0103
i: 10000, loss: 0.0015
i: 15000, loss: 0.0005
i: 20000, loss: 0.0004


In [5]:
print('input:\n',x_in)
print('output:\n',xor_nn(x_in))

input:
 Variable containing:
 1  1
 0  0
 1  0
 0  1
[torch.FloatTensor of size 4x2]

output:
 Variable containing:
 0.0293
 0.0313
 0.9738
 0.9698
[torch.FloatTensor of size 4x1]



In [6]:
# weights in the model
xor_nn.state_dict()

OrderedDict([('l1_linear.weight', 
              -2.6394  5.9109
              -3.8154 -4.0501
              -6.1087  2.8830
               3.0791  3.3121
              [torch.FloatTensor of size 4x2]), ('l2_linear.weight', 
               -8.7516 -11.1080   8.3685   4.6243
              [torch.FloatTensor of size 1x4])])